## Toxic comments - préparation des données

1. Créer une variable cible binaire
2. Sous-échantillonnage
3. Nettoyage du texte
4. Séparer l'entraînement et l'ensemble de test
5. Représentation de texte Tf-Idf

### 1. Céer une variable cible binaire

In [1]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('train.csv')
train.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
def flag_df(train):

    if ((train.toxic == 1) | (train.severe_toxic == 1) | (train.obscene == 1) | (train.threat ==1) | (train.insult ==1) | (train.identity_hate == 1)):
        return 1
    else:
        return 0
    
train['tox'] = train.apply(flag_df, axis = 1)

In [6]:
df= train[["id", "comment_text", "tox"]]
df[df.tox== 0].describe()


,tox
count,143346.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [8]:
# nom de variable: tox
# 0 commentaire normale, 1 commentarie toxic

### 2. Sous-échantillonnage


In [7]:
normal= df[df.tox==0]
normal.describe()


,tox
count,143346.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [8]:
toxic= df[df.tox==1]
toxic.describe()

,tox
count,16225.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [9]:
#subsampe normal comments to 16 225

normal_sub= normal.sample(n=16225, replace=False, random_state=345)
normal_sub.describe()

,tox
count,16225.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [10]:
#append toxic and normal subsampled data

df_balanced = toxic.append(normal_sub)
df_balanced.describe()


,tox
count,32450.000000
mean,0.500000
std,0.500008
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [11]:
df_balanced.to_csv('ensemble_equilibré.csv', index=False) 

In [86]:
import pandas as pd
import numpy as np

In [13]:
train = pd.read_csv('ensemble_equilibré.csv')
train.head()

,id,comment_text,tox
0,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
1,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1
2,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1
3,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1
4,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1


### 3. Nettoyage du texte

In [87]:
#3.1 Lowercase
df_balanced = pd.read_csv('ensemble_equilibré.csv')

df_balanced['comment_text'] = df_balanced['comment_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_balanced.head()

,id,comment_text,tox
0,0002bcb3da6cb337,cocksucker before you piss around on my work,1
1,0005c987bdfc9d4b,hey... what is it.. @ | talk . what is it... a...,1
2,0007e25b2121310b,"bye! don't look, come or think of comming back...",1
3,001810bf8c45bf5f,you are gay or antisemmitian? archangel white ...,1
4,00190820581d90ce,"fuck your filthy mother in the ass, dry!",1


In [88]:
#Remove URLs 

df_balanced['comment_text'] = df_balanced['comment_text'].str.replace('[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)','')
df_balanced.head()

,id,comment_text,tox
0,0002bcb3da6cb337,cocksucker before you piss around on my work,1
1,0005c987bdfc9d4b,hey... what is it.. @ | talk . what is it... a...,1
2,0007e25b2121310b,"bye! don't look, come or think of comming back...",1
3,001810bf8c45bf5f,you are gay or antisemmitian? archangel white ...,1
4,00190820581d90ce,"fuck your filthy mother in the ass, dry!",1


In [89]:
#Remove punctuation

df_balanced['comment_text'] = df_balanced['comment_text'].str.replace('[^\w\s]','')
df_balanced.head()

,id,comment_text,tox
0,0002bcb3da6cb337,cocksucker before you piss around on my work,1
1,0005c987bdfc9d4b,hey what is it talk what is it an exclusive...,1
2,0007e25b2121310b,bye dont look come or think of comming back to...,1
3,001810bf8c45bf5f,you are gay or antisemmitian archangel white t...,1
4,00190820581d90ce,fuck your filthy mother in the ass dry,1


In [90]:
#correct for _ and numbers

df_balanced['comment_text'] = df_balanced['comment_text'].str.replace('_',' ')
df_balanced['comment_text'] = df_balanced['comment_text'].str.replace('\d','')
df_balanced.head()

,id,comment_text,tox
0,0002bcb3da6cb337,cocksucker before you piss around on my work,1
1,0005c987bdfc9d4b,hey what is it talk what is it an exclusive...,1
2,0007e25b2121310b,bye dont look come or think of comming back to...,1
3,001810bf8c45bf5f,you are gay or antisemmitian archangel white t...,1
4,00190820581d90ce,fuck your filthy mother in the ass dry,1


In [91]:
df_balanced.head()

,id,comment_text,tox
0,0002bcb3da6cb337,cocksucker before you piss around on my work,1
1,0005c987bdfc9d4b,hey what is it talk what is it an exclusive...,1
2,0007e25b2121310b,bye dont look come or think of comming back to...,1
3,001810bf8c45bf5f,you are gay or antisemmitian archangel white t...,1
4,00190820581d90ce,fuck your filthy mother in the ass dry,1


In [92]:
#Remove stopwords
import nltk
from nltk.corpus import stopwords

stop= nltk.download('stopwords')
stop = stopwords.words('english')
df_balanced['comment_text'] = df_balanced['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df_balanced.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\babet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,comment_text,tox
0,0002bcb3da6cb337,cocksucker piss around work,1
1,0005c987bdfc9d4b,hey talk exclusive group wp talibanswho good d...,1
2,0007e25b2121310b,bye dont look come think comming back tosser,1
3,001810bf8c45bf5f,gay antisemmitian archangel white tiger meow g...,1
4,00190820581d90ce,fuck filthy mother ass dry,1


In [93]:
# remove the 10 most frequent (but keep the first one!)

dellist = pd.Series(' '.join(df_balanced['comment_text']).split()).value_counts()[1:10]
print(dellist)

article      7527
page         7195
wikipedia    7078
like         6457
dont         6420
talk         4807
one          4756
please       4264
im           4243
dtype: int64


In [94]:
dellist = list(dellist.index)
df_balanced['comment_text'] = df_balanced['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in dellist))
df_balanced.head()

,id,comment_text,tox
0,0002bcb3da6cb337,cocksucker piss around work,1
1,0005c987bdfc9d4b,hey exclusive group wp talibanswho good destro...,1
2,0007e25b2121310b,bye look come think comming back tosser,1
3,001810bf8c45bf5f,gay antisemmitian archangel white tiger meow g...,1
4,00190820581d90ce,fuck filthy mother ass dry,1


In [95]:
#rare words removal

freq = pd.Series(' '.join(df_balanced['comment_text']).split()).value_counts()[-40976:]
#freq = pd.Series(' '.join(df_balanced['comment_text']).split()).value_counts()[-100:]

print(freq)

projewish                                                      1
talkdawnseekerviewsuser                                        1
butyou                                                         1
chitchatting                                                   1
fps                                                            1
margereat                                                      1
madea                                                          1
recurrent                                                      1
importnt                                                       1
bushkissers                                                    1
sushi                                                          1
neuteral                                                       1
heraldic                                                       1
hiii                                                           1
mixtape                                                        1
donohuehtml              

In [96]:
freq = list(freq.index)
df_balanced['comment_text'] = df_balanced['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df_balanced.head()

KeyboardInterrupt: 

In [97]:
from textblob import Word
df_balanced['comment_text'] = df_balanced['comment_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df_balanced['comment_text'].head(170)

0                            cocksucker piss around work
1      hey exclusive group wp talibanswho good destro...
2                bye look come think comming back tosser
3      gay antisemmitian archangel white tiger meow g...
4                              fuck filthy mother as dry
5      sorry sorry screwed around someone bad know te...
6      get fucked get fuckeeed got drink cant put get...
7      stupid peace shit stop deleting stuff asshole ...
8          tony sidaway obviously fistfuckee love arm as
9      band page deletion thought gone deleting comme...
10     cant believe fat artie see recent appearence t...
11     edits good cunt revert good edits youre stupid...
12           hi back last warning stop undoing edits die
13                  would shut run especially stupid kid
14                    pair jewhating weiner nazi schmuck
15     sorry puck ever said dick number clearly liste...
16     unblock ill get lawyer blocking constitutional...
17     fired youre moronic wimp

In [100]:
df_balanced['comment_text'].head(3662)

0                             cocksucker piss around work
1       hey exclusive group wp talibanswho good destro...
2                 bye look come think comming back tosser
3       gay antisemmitian archangel white tiger meow g...
4                               fuck filthy mother as dry
5       sorry sorry screwed around someone bad know te...
6       get fucked get fuckeeed got drink cant put get...
7       stupid peace shit stop deleting stuff asshole ...
8           tony sidaway obviously fistfuckee love arm as
9       band page deletion thought gone deleting comme...
10      cant believe fat artie see recent appearence t...
11      edits good cunt revert good edits youre stupid...
12            hi back last warning stop undoing edits die
13                   would shut run especially stupid kid
14                     pair jewhating weiner nazi schmuck
15      sorry puck ever said dick number clearly liste...
16      unblock ill get lawyer blocking constitutional...
17      fired 

In [101]:
#delete null values:

null_val= np.where(df_balanced.applymap(lambda x: x == ''))
print(null_val[0])


df_balanced= df_balanced.drop([3661, 16344, 16660, 18882, 21149, 23057, 30605, 30623, 30690, 32323])


[ 3661 16344 16660 18882 21149 23057 30605 30623 30690 32323]


### 4. Séparer l'entraînement et l'ensemble de test

In [102]:
from sklearn import model_selection, preprocessing
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_balanced['comment_text'], df_balanced['tox'], random_state= 43)


In [105]:
valid_x.describe()

count     8110
unique    8094
top       fuck
freq         5
Name: comment_text, dtype: object

In [108]:
train_x.to_csv('train_x.csv', index= None) 
valid_x.to_csv('test_x.csv', index= None) 
train_y.to_csv('train_class.csv', index= None) 
valid_y.to_csv('test_class.csv', index= None) 

C:\Users\babet\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
C:\Users\babet\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
C:\Users\babet\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\U

### 5. Représentation de texte Tf-Idf

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',max_features=10000)
#max_features=5000
tfidf_vect.fit(df_balanced['comment_text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [64]:
type(xtrain_tfidf)

scipy.sparse.csr.csr_matrix

In [67]:
xtrain_tfidf= pd.DataFrame(xtrain_tfidf.todense())
xvalid_tfidf= pd.DataFrame(xvalid_tfidf.todense())

In [68]:
xvalid_tfidf.head(400)

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
xtrain_tfidf.to_csv('train_tfidf_m.csv', index=False) 
xvalid_tfidf.to_csv('test_tfidf_m.csv', index=False) 
train_y.to_csv('train_class_m.csv', index=False) 
valid_y.to_csv('test_class_m.csv', index=False) 

OSError: [Errno 28] No space left on device

In [69]:
type(valid_y)

train_xx = xtrain_tfidf
col_list= list(train_xx)
col_list



train_xx['e'] = train_xx[col_list].sum(axis=1)
print(len(train_xx[train_xx['e'] == 0.000000]))

55
